<a href="https://colab.research.google.com/github/shubhangkhare/Transformers/blob/main/12.2%20Split%20model.safetensors%20into%20n%20parts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from transformers import pipeline

pipe = pipeline("token-classification", model="dbmdz/bert-large-cased-finetuned-conll03-english")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

In [2]:
pipe('My name is Wolfgang and I live in Berlin')

[{'entity': 'I-PER',
  'score': 0.99914825,
  'index': 4,
  'word': 'Wolfgang',
  'start': 11,
  'end': 19},
 {'entity': 'I-LOC',
  'score': 0.9983669,
  'index': 9,
  'word': 'Berlin',
  'start': 34,
  'end': 40}]

In [3]:
pipe.save_pretrained('/content/bert/')

In [5]:
import os

def split_file(input_file, num_parts, output_folder):
    # Read the input file
    with open(input_file, 'rb') as f:
        data = f.read()

    # Determine the size of each part
    total_size = len(data)
    part_size = total_size // num_parts

    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Split and write the parts
    for i in range(num_parts):
        start_index = i * part_size
        end_index = (i + 1) * part_size if i < num_parts - 1 else total_size
        part_data = data[start_index:end_index]
        part_file = os.path.join(output_folder, f'part_{i + 1}.safetensors')

        with open(part_file, 'wb') as f_part:
            f_part.write(part_data)

# Usage
split_file('/content/bert/model.safetensors', 3, '/content/model_parts')

In [6]:
def join_files(input_folder, output_file):
    # List all part files in the input folder
    part_files = sorted([f for f in os.listdir(input_folder) if f.startswith('part_') and f.endswith('.safetensors')],
                        key=lambda x: int(x.split('_')[1].split('.')[0]))

    # Read and concatenate all parts
    data = bytearray()
    for part_file in part_files:
        with open(os.path.join(input_folder, part_file), 'rb') as f_part:
            data.extend(f_part.read())

    # Write the combined data to the output file
    with open(output_file, 'wb') as f:
        f.write(data)

# Usage
join_files('/content/model_parts', '/content/bert/model.safetensors')

In [7]:
pipe = pipeline("token-classification", model="/content/bert/")

In [8]:
pipe('My name is Wolfgang and I live in Berlin')

[{'entity': 'I-PER',
  'score': 0.99914825,
  'index': 4,
  'word': 'Wolfgang',
  'start': 11,
  'end': 19},
 {'entity': 'I-LOC',
  'score': 0.9983669,
  'index': 9,
  'word': 'Berlin',
  'start': 34,
  'end': 40}]